In [1]:
from Bio import SeqIO

import numpy as np
import pandas as pd
from bisect import bisect

from tools import calculate_kmer_features, signs
import tools

np.random.seed(42)

Файл 'chromosome1.fasta' содержит **последовательность нуклеотидов** первой хромосомы человека

In [2]:
sequence_data = 'chromosome1.fasta'
with open(sequence_data) as file:
    fasta_sequences = SeqIO.parse(file, 'fasta')
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)

Импорт данных 

In [3]:
cds = pd.read_csv('data/cds.csv')
exons = pd.read_csv('data/exons.csv')

In [4]:
exons.head()

,Seqid,Source,Type,Start,End,Score,Strand,Phase
0,NC_000001.11,BestRefSeq,exon,11874,12227,NaN,+,NaN
1,NC_000001.11,BestRefSeq,exon,12613,12721,NaN,+,NaN
2,NC_000001.11,BestRefSeq,exon,13221,14409,NaN,+,NaN
3,NC_000001.11,BestRefSeq,exon,30366,30503,NaN,+,NaN
4,NC_000001.11,BestRefSeq,exon,30438,30458,NaN,+,NaN


Выделение границ экзонов и кодирующих последовательностей

In [5]:
cds_starts = np.sort(list(set(cds['Start'].values)))
cds_ends = np.sort(list(set(cds['End'].values)))

exons_starts = np.sort(list(set(exons['Start'].values)))
exons_ends = np.sort(list(set(exons['Start'].values)))

Из всех cds нас интересуют начальные (начинаются с **ATG**) и конечные (кончаются на **TAA, TAG, TGA**/)

In [6]:
cds_starts_new = []
cds_ends_new = []
    
for start, end in zip(cds_starts, cds_ends):
    
    if (sequence[start-1:start+2] in ['ATG']): 
        cds_starts_new.append(start)
    if (sequence[end-3:end] in ['TAA', 'TAG', 'TGA']):  
        cds_ends_new.append(end)

Чтобы определить последовательность перед cds, необходимо локализовать ближайший слева экзон

In [7]:
seq_before_cds_location = []
for cds_start in cds_starts_new:
    p = bisect(exons_starts, cds_start)
    seq_before_cds_location.append((exons_starts[p-1], cds_start))
seq_before_cds_location = np.array(seq_before_cds_location)

На выходе имеем массив с границами участков перед cds внутри соответствующих экзонов

In [8]:
seq_before_cds_location

array([[    65520,     65565],
       [   923923,    924432],
       [   925922,    925942],
       ...,
       [237678048, 237678048],
       [239907433, 239907452],
       [240091883, 240092110]], dtype=int64)

Выберем последовательности длиннее **50 нуклеотидов**, чтобы рассчитываемые далее статистики были релевантны

In [9]:
mask = np.diff(seq_before_cds_location)>50
seq_before_cds_location = seq_before_cds_location[np.tile(mask, (1, 2))].reshape(-1, 2)-1

cds_location_data = pd.DataFrame(seq_before_cds_location, columns=['start', 'end'])
cds_location_data.to_csv('data/cds_location_data.csv', index=False)

Используя список границ областей интереса, вырежем эти области из исходной последовательности

In [10]:
seq_before_cds = []
for i, j in seq_before_cds_location:
    seq_before_cds.append(sequence[i:j])

Далее необходимо создать аналогичную **выборку для класса 0**, то есть последовательностей, которые **не расположены** перед cds  
Для этого выбираем случайный нуклеотид в цепочке и вырезаем последовательность **длинной N**  
**Число N** определяется из **экспоненциального распределения**, близкого к распределению длин искомых последовательностей  
Дополнительно мы убеждаемся, что выбранные последовательности класса 0 **не пересекаются** с границами последовательностей класса 1

In [11]:
N = 5
seq_zero_location = []
for sample in range(int(N*seq_before_cds_location.shape[0])):
    b = np.random.randint(1e4, 2e8)
    bounds = (b, b+int(np.random.exponential(scale=200.0)+50))
    
    flag = 0
    for bounds2 in seq_before_cds_location:
        r = max(bounds[0], bounds2[0]), min(bounds[1]+1, bounds2[1]+1)
        if set(range(*r))!=set():
            flag = 1
            break
            
    if flag==0: 
        seq_zero_location.append(bounds)

Ниже можно убедиться, что <1% примеров класса 0 пересекались с примерами класса 1 и были исключены

In [12]:
len(seq_zero_location)/(N*seq_before_cds_location.shape[0])

0.9979746835443037

Используя полученные границы последдовательностей класса 0, вырезаем их из исходной последовательности

In [13]:
seq_zero = []
for i, j in seq_zero_location:
    seq_zero.append(sequence[i:j])

По полученным кусочкам последовательности рассчитываем признаки используя функцию **calculate_kmer_features**

In [14]:
start_data = pd.DataFrame(columns=signs)
zero_data = pd.DataFrame(columns=signs)

for cnt, seq in enumerate(seq_before_cds):
    start_data.loc[cnt] = calculate_kmer_features(seq, signs)['Entropy'].values
    
for cnt, seq in enumerate(seq_zero):
    zero_data.loc[cnt] = calculate_kmer_features(seq, signs)['Entropy'].values

In [15]:
start_data.head()

,A,T,G,C,AT,GC,GA,GT,GG,CG,...,GAC,GGC,GTA,GCA,GAA,GGA,GTG,GCG,GAG,GGG
0,0.082616,0.081422,0.210323,0.188503,0.014111,0.119088,0.055010,0.032336,0.108594,0.115360,...,0.026727,0.077779,0.003623,0.011733,0.006557,0.032336,0.016389,0.070187,0.024772,0.056468
1,0.108048,0.088818,0.189905,0.189905,0.013003,0.103477,0.060316,0.060316,0.066524,0.112478,...,0.013003,0.039512,0.000000,0.031580,0.013003,0.013003,0.022881,0.053784,0.039512,0.039512
2,0.136355,0.107077,0.169011,0.184400,0.017491,0.107077,0.052026,0.041811,0.061423,0.041811,...,0.000000,0.052026,0.017491,0.017491,0.017491,0.017491,0.030497,0.030497,0.030497,0.000000
3,0.024356,0.104104,0.167360,0.233368,0.000000,0.139359,0.000000,0.041956,0.041956,0.139359,...,0.000000,0.041956,0.000000,0.000000,0.000000,0.000000,0.000000,0.057005,0.000000,0.000000
4,0.099628,0.100578,0.186891,0.197117,0.015095,0.101522,0.062778,0.020999,0.084565,0.081348,...,0.022858,0.045398,0.000000,0.024672,0.010794,0.037939,0.015095,0.029880,0.037939,0.043948


In [16]:
zero_data.head()

,A,T,G,C,AT,GC,GA,GT,GG,CG,...,GAC,GGC,GTA,GCA,GAA,GGA,GTG,GCG,GAG,GGG
0,0.182262,0.160113,0.134521,0.130625,0.066887,0.042952,0.053798,0.053798,0.046684,0.018559,...,0.009502,0.023411,0.013313,0.018559,0.024959,0.013313,0.026478,0.005287,0.018559,0.013313
1,0.135977,0.207140,0.126121,0.129899,0.074384,0.022250,0.060839,0.058783,0.035953,0.000000,...,0.016010,0.005009,0.019203,0.000000,0.025175,0.016010,0.012634,0.000000,0.019203,0.012634
2,0.135977,0.178105,0.158778,0.135977,0.049536,0.058539,0.039768,0.066931,0.066931,0.016586,...,0.028970,0.039768,0.039768,0.049536,0.000000,0.016586,0.016586,0.000000,0.000000,0.028970
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.157457,0.176033,0.124387,0.147861,0.078380,0.038903,0.052980,0.033100,0.049622,0.012789,...,0.017839,0.015369,0.010068,0.007160,0.020216,0.031084,0.010068,0.007160,0.015369,0.017839


Дополним таблицы колонкой **target** и объединим в одну

In [17]:
start_data.loc[:, 'target'] = 1
zero_data.loc[:, 'target'] = 0
data = pd.concat([start_data, zero_data], axis=0)
data.head()

,A,T,G,C,AT,GC,GA,GT,GG,CG,...,GGC,GTA,GCA,GAA,GGA,GTG,GCG,GAG,GGG,target
0,0.082616,0.081422,0.210323,0.188503,0.014111,0.119088,0.055010,0.032336,0.108594,0.115360,...,0.077779,0.003623,0.011733,0.006557,0.032336,0.016389,0.070187,0.024772,0.056468,1
1,0.108048,0.088818,0.189905,0.189905,0.013003,0.103477,0.060316,0.060316,0.066524,0.112478,...,0.039512,0.000000,0.031580,0.013003,0.013003,0.022881,0.053784,0.039512,0.039512,1
2,0.136355,0.107077,0.169011,0.184400,0.017491,0.107077,0.052026,0.041811,0.061423,0.041811,...,0.052026,0.017491,0.017491,0.017491,0.017491,0.030497,0.030497,0.030497,0.000000,1
3,0.024356,0.104104,0.167360,0.233368,0.000000,0.139359,0.000000,0.041956,0.041956,0.139359,...,0.041956,0.000000,0.000000,0.000000,0.000000,0.000000,0.057005,0.000000,0.000000,1
4,0.099628,0.100578,0.186891,0.197117,0.015095,0.101522,0.062778,0.020999,0.084565,0.081348,...,0.045398,0.000000,0.024672,0.010794,0.037939,0.015095,0.029880,0.037939,0.043948,1


На этом предобработка завершена ;)

In [18]:
data.to_csv('data/data_for_ml_model.csv', index=False)